# Construction Professionnelle de Graphe d'Articles Scientifiques

**Objectif**: Construire un graphe d'articles optimisé pour GNN et analyse de communautés

## Pipeline:
1. Chargement des embeddings et métadonnées
2. Construction du graphe k-NN avec poids de similarité
3. Détection des communautés (Louvain)
4. Analyse topologique avancée
5. Export pour GNN (PyTorch Geometric format)
6. Visualisations professionnelles

## 📦 Imports et Configuration

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from pathlib import Path
from typing import Tuple, Dict, List, Optional
from dataclasses import dataclass
import pickle
import json

# Machine Learning
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from community import community_louvain

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Configuration
import warnings
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

print("✅ Imports réussis")

✅ Imports réussis


In [2]:
# Configuration du projet
BASE_PATH = Path("../data/processed")
OUTPUT_PATH = BASE_PATH / "graph_outputs"
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# Fichiers d'entrée
EMBEDDINGS_PATH = BASE_PATH / "embeddings.npy"
ARTICLES_PATH = BASE_PATH / "cleaned_articles.csv"

# Paramètres du graphe
K_NEIGHBORS = 10
SIMILARITY_THRESHOLD = 0.2
USE_COSINE = True
SAMPLE_SIZE = None  # None = tout le dataset

print(f"📂 Répertoire de sortie: {OUTPUT_PATH}")
print(f"🔧 K-voisins: {K_NEIGHBORS}")
print(f"🔧 Seuil de similarité: {SIMILARITY_THRESHOLD}")

📂 Répertoire de sortie: ..\data\processed\graph_outputs
🔧 K-voisins: 10
🔧 Seuil de similarité: 0.2


## 📂 1. Chargement des Données

In [3]:
print("\n" + "="*70)
print("📂 CHARGEMENT DES DONNÉES")
print("="*70)

# Charger embeddings
print(f"\n🔄 Chargement des embeddings...")
if not EMBEDDINGS_PATH.exists():
    raise FileNotFoundError(f"Embeddings introuvables: {EMBEDDINGS_PATH}")

embeddings = np.load(EMBEDDINGS_PATH)
print(f"   ✅ Shape: {embeddings.shape}")
print(f"   ✅ Type: {embeddings.dtype}")
print(f"   ✅ Taille mémoire: {embeddings.nbytes / 1024**2:.2f} MB")

# Charger articles
print(f"\n🔄 Chargement des articles...")
if not ARTICLES_PATH.exists():
    raise FileNotFoundError(f"Articles introuvables: {ARTICLES_PATH}")

articles_df = pd.read_csv(ARTICLES_PATH)
print(f"   ✅ {len(articles_df):,} articles chargés")
print(f"   ✅ Colonnes: {list(articles_df.columns[:5])}...")

# Synchroniser tailles
n_samples = min(len(articles_df), embeddings.shape[0])
articles_df = articles_df.iloc[:n_samples].reset_index(drop=True)
embeddings = embeddings[:n_samples]

# Échantillonnage optionnel
if SAMPLE_SIZE and n_samples > SAMPLE_SIZE:
    print(f"\n⚠️  Échantillonnage: {SAMPLE_SIZE} articles")
    indices = np.random.RandomState(42).choice(n_samples, SAMPLE_SIZE, replace=False)
    articles_df = articles_df.iloc[indices].reset_index(drop=True)
    embeddings = embeddings[indices]

print(f"\n✅ Données finales: {len(articles_df):,} articles, dim={embeddings.shape[1]}")


📂 CHARGEMENT DES DONNÉES

🔄 Chargement des embeddings...
   ✅ Shape: (816359, 384)
   ✅ Type: float32
   ✅ Taille mémoire: 1195.84 MB

🔄 Chargement des articles...
   ✅ 816,359 articles chargés
   ✅ Colonnes: ['cord_uid', 'sha', 'source_x', 'title', 'doi']...

✅ Données finales: 816,359 articles, dim=384


In [4]:
# Aperçu des données
print("\n📊 Aperçu des articles:")
display(articles_df[['title', 'abstract']].head(3))

print("\n📊 Statistiques des embeddings:")
print(f"   Moyenne: {embeddings.mean():.4f}")
print(f"   Std: {embeddings.std():.4f}")
print(f"   Min: {embeddings.min():.4f}")
print(f"   Max: {embeddings.max():.4f}")


📊 Aperçu des articles:


,title,abstract
0,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...
1,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...
2,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...



📊 Statistiques des embeddings:
   Moyenne: -0.0008
   Std: 0.0510
   Min: -0.2988
   Max: 0.2857


## 🔗 2. Construction du Graphe k-NN

In [ ]:
# ============================================================================
# CODE OPTIMISÉ À REMPLACER DANS LE NOTEBOOK (SANS FAISS)
# Section: Construction du Graphe k-NN
# Optimisations: batch processing, sparse graph, memory efficient
# ============================================================================

from sklearn.neighbors import NearestNeighbors
from scipy.sparse import lil_matrix, csr_matrix
import gc

print("\n" + "="*70)
print("🔗 CONSTRUCTION DU GRAPHE k-NN OPTIMISÉE (sklearn)")
print("="*70)

# ========================
# PARAMÈTRES OPTIMISÉS
# ========================
K_NEIGHBORS = 10
SIMILARITY_THRESHOLD = 0.35  # Plus élevé = graphe plus sparse = plus rapide
BATCH_SIZE = 20000  # Batch size pour la recherche
N_JOBS = -1  # Utiliser tous les CPU disponibles

# Option 1: Échantillonnage pour test rapide (recommandé pour 800K articles)
# SAMPLE_SIZE = 100000  # Décommenter pour tester sur 100K articles d'abord
SAMPLE_SIZE = None  # None = tous les articles (attention: très lent!)

print(f"\n🔧 Configuration:")
print(f"   Articles: {len(articles_df):,}")
print(f"   K-voisins: {K_NEIGHBORS}")
print(f"   Seuil: {SIMILARITY_THRESHOLD}")
print(f"   Batch size: {BATCH_SIZE:,}")
print(f"   CPU cores: tous disponibles")
if SAMPLE_SIZE:
    print(f"   ⚠️  MODE ÉCHANTILLON: {SAMPLE_SIZE:,} articles")

# ========================
# ÉCHANTILLONNAGE (optionnel mais recommandé)
# ========================
if SAMPLE_SIZE and len(embeddings) > SAMPLE_SIZE:
    print(f"\n📊 Échantillonnage de {SAMPLE_SIZE:,} articles...")
    np.random.seed(42)
    sample_indices = np.random.choice(len(embeddings), SAMPLE_SIZE, replace=False)
    sample_indices = np.sort(sample_indices)  # Garder l'ordre
    
    embeddings = embeddings[sample_indices]
    articles_df = articles_df.iloc[sample_indices].reset_index(drop=True)
    
    print(f"   ✅ Échantillon créé: {len(embeddings):,} articles")

n_samples = len(embeddings)

# ========================
# NORMALISATION
# ========================
print(f"\n🔧 Normalisation des embeddings...")
norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
norms[norms == 0] = 1.0
embeddings_normalized = embeddings / norms
print(f"   ✅ Normes moyennes: {np.linalg.norm(embeddings_normalized, axis=1).mean():.4f}")

# ========================
# OPTION A: Construction par batch (RECOMMANDÉ pour 800K articles)
# ========================
if n_samples > 50000:
    print(f"\n🚀 MODE BATCH (dataset large: {n_samples:,} articles)")
    print(f"   Construction de l'index k-NN une seule fois...")
    
    # Construire l'index une fois
    nn_model = NearestNeighbors(
        n_neighbors=K_NEIGHBORS + 1,
        metric='cosine',
        algorithm='brute',  # Plus rapide pour cosine sur datasets moyens
        n_jobs=N_JOBS
    )
    nn_model.fit(embeddings_normalized)
    print(f"   ✅ Index k-NN construit")
    
    # Recherche par batch pour économiser mémoire
    print(f"\n🔍 Recherche k-NN par batch de {BATCH_SIZE:,}...")
    
    all_distances = []
    all_indices = []
    
    for i in tqdm(range(0, n_samples, BATCH_SIZE), desc="k-NN batch search"):
        end_idx = min(i + BATCH_SIZE, n_samples)
        batch = embeddings_normalized[i:end_idx]
        
        # Recherche pour ce batch
        distances, indices = nn_model.kneighbors(batch, return_distance=True)
        
        all_distances.append(distances)
        all_indices.append(indices)
        
        # Libérer mémoire toutes les 5 batchs
        if (i // BATCH_SIZE) % 5 == 0:
            gc.collect()
    
    # Concaténer tous les résultats
    distances = np.vstack(all_distances)
    indices = np.vstack(all_indices)
    
    # Convertir en similarités
    similarities = 1.0 - distances
    
    # Libérer mémoire
    del all_distances, all_indices, nn_model
    gc.collect()

# ========================
# OPTION B: Construction directe (pour datasets < 50K)
# ========================
else:
    print(f"\n🚀 MODE DIRECT (dataset petit: {n_samples:,} articles)")
    
    nn_model = NearestNeighbors(
        n_neighbors=K_NEIGHBORS + 1,
        metric='cosine',
        algorithm='auto',
        n_jobs=N_JOBS
    )
    nn_model.fit(embeddings_normalized)
    
    print(f"   Recherche k-NN...")
    distances, indices = nn_model.kneighbors(embeddings_normalized, return_distance=True)
    similarities = 1.0 - distances
    
    del nn_model
    gc.collect()

print(f"\n✅ k-NN terminé")
print(f"   Similarité moyenne: {similarities[:, 1:].mean():.4f}")
print(f"   Similarité min: {similarities[:, 1:].min():.4f}")
print(f"   Similarité max: {similarities[:, 1:].max():.4f}")

# Libérer mémoire
del embeddings_normalized
gc.collect()

# ========================
# CONSTRUCTION DES ARÊTES (OPTIMISÉE avec matrice sparse)
# ========================
print(f"\n🏗️  Construction des arêtes (seuil={SIMILARITY_THRESHOLD})...")
print(f"   Utilisation de matrice sparse pour efficacité mémoire...")

# Utiliser une matrice sparse pour construction plus rapide
edge_matrix = lil_matrix((n_samples, n_samples), dtype=np.float32)
edge_count = 0

for i in tqdm(range(indices.shape[0]), desc="Building edges", mininterval=1.0):
    for j in range(1, indices.shape[1]):  # Skip self (j=0)
        neighbor_idx = int(indices[i, j])
        similarity = float(similarities[i, j])
        
        # Filtrer par seuil
        if similarity >= SIMILARITY_THRESHOLD:
            # Éviter doublons (stocker seulement i < j)
            if i < neighbor_idx:
                edge_matrix[i, neighbor_idx] = similarity
                edge_count += 1
            elif i > neighbor_idx:
                edge_matrix[neighbor_idx, i] = similarity
                edge_count += 1
    
    # Affichage périodique
    if (i + 1) % 50000 == 0:
        print(f"      {edge_count:,} arêtes créées jusqu'à présent...")
        gc.collect()

# Convertir en format CSR (plus efficace pour NetworkX)
edge_matrix = edge_matrix.tocsr()

print(f"\n✅ {edge_count:,} arêtes uniques créées")
print(f"   Arêtes par nœud (moyenne): {edge_count / n_samples:.2f}")

# Vérifier si le graphe n'est pas trop sparse
if edge_count / n_samples < 2:
    print(f"   ⚠️  Graphe très sparse! Réduisez SIMILARITY_THRESHOLD à 0.25-0.3")
elif edge_count / n_samples > 50:
    print(f"   ⚠️  Graphe très dense! Augmentez SIMILARITY_THRESHOLD à 0.4-0.5")

# Libérer mémoire
del distances, indices, similarities
gc.collect()

# ========================
# CRÉATION DU GRAPHE NETWORKX (OPTIMISÉE)
# ========================
print(f"\n🔨 Création du graphe NetworkX...")

# Créer graphe vide
G = nx.Graph()

# Ajouter nœuds avec attributs (optimisé)
print(f"   Ajout de {n_samples:,} nœuds...")
node_data = [
    (i, {
        'title': str(row.get('title', ''))[:150],
        'source': str(row.get('source_x', ''))[:50]
    })
    for i, row in articles_df.iterrows()
]
G.add_nodes_from(node_data)
del node_data
gc.collect()

# Ajouter arêtes depuis la matrice sparse
print(f"   Ajout de {edge_count:,} arêtes...")
edges_to_add = []

# Extraire les arêtes de la matrice sparse
rows, cols = edge_matrix.nonzero()
for idx in tqdm(range(len(rows)), desc="Extracting edges", mininterval=1.0):
    i, j = rows[idx], cols[idx]
    weight = edge_matrix[i, j]
    edges_to_add.append((int(i), int(j), float(weight)))
    
    # Ajouter par batch de 50K pour éviter OOM
    if len(edges_to_add) >= 50000:
        G.add_weighted_edges_from(edges_to_add)
        edges_to_add = []
        gc.collect()

# Ajouter les arêtes restantes
if edges_to_add:
    G.add_weighted_edges_from(edges_to_add)

del edge_matrix, edges_to_add
gc.collect()

print(f"\n✅ Graphe créé:")
print(f"   Nœuds: {G.number_of_nodes():,}")
print(f"   Arêtes: {G.number_of_edges():,}")
print(f"   Densité: {nx.density(G):.8f}")

# ========================
# STATISTIQUES RAPIDES
# ========================
print(f"\n📊 Statistiques rapides:")
degrees = dict(G.degree())
degree_values = list(degrees.values())

print(f"   Degré moyen: {np.mean(degree_values):.2f}")
print(f"   Degré médian: {np.median(degree_values):.0f}")
print(f"   Degré min: {min(degree_values)}")
print(f"   Degré max: {max(degree_values)}")

# Nœuds isolés
isolated = [n for n in G.nodes() if G.degree(n) == 0]
if isolated:
    print(f"   ⚠️  Nœuds isolés: {len(isolated)} ({len(isolated)/n_samples*100:.2f}%)")
    print(f"       Considérez réduire SIMILARITY_THRESHOLD")

print(f"\n💾 Mémoire libérée, prêt pour la détection de communautés...")

# ========================
# CONSEIL POUR ACCÉLÉRER ENCORE PLUS
# ========================
print(f"\n💡 CONSEIL PERFORMANCE:")
print(f"   Pour 800K+ articles, considérez:")
print(f"   1. Échantillonner 100K-200K articles (décommenter SAMPLE_SIZE)")
print(f"   2. Augmenter SIMILARITY_THRESHOLD à 0.4-0.5")
print(f"   3. Réduire K_NEIGHBORS à 5-8")
print(f"   4. Ou installer FAISS: pip install faiss-cpu")


🔗 CONSTRUCTION DU GRAPHE k-NN OPTIMISÉE (sklearn)

🔧 Configuration:
   Articles: 100,000
   K-voisins: 10
   Seuil: 0.35
   Batch size: 20,000
   CPU cores: tous disponibles

🔧 Normalisation des embeddings...
   ✅ Normes moyennes: 1.0000

🚀 MODE BATCH (dataset large: 100,000 articles)
   Construction de l'index k-NN une seule fois...
   ✅ Index k-NN construit

🔍 Recherche k-NN par batch de 20,000...


k-NN batch search:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
print("\n" + "="*70)
print("🔗 CONSTRUCTION DU GRAPHE k-NN")
print("="*70)

# Normaliser les embeddings pour similarité cosinus
if USE_COSINE:
    print("\n🔧 Normalisation des embeddings (cosine similarity)...")
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    embeddings_normalized = embeddings / norms
    print(f"   ✅ Normes moyennes: {np.linalg.norm(embeddings_normalized, axis=1).mean():.4f}")
else:
    embeddings_normalized = embeddings

# Construire l'index k-NN
print(f"\n🔄 Construction de l'index k-NN (k={K_NEIGHBORS})...")
metric = "cosine" if USE_COSINE else "euclidean"
nn_model = NearestNeighbors(n_neighbors=K_NEIGHBORS + 1, metric=metric, n_jobs=-1)
nn_model.fit(embeddings_normalized)
print(f"   ✅ Index k-NN construit (metric={metric})")

# Recherche des voisins
print(f"\n🔍 Recherche des {K_NEIGHBORS} plus proches voisins...")
distances, indices = nn_model.kneighbors(embeddings_normalized, return_distance=True)

# Calculer les poids (similarités)
if USE_COSINE:
    weights = 1.0 - distances  # Cosine similarity
else:
    weights = 1.0 / (1.0 + distances)  # Inverse distance

print(f"   ✅ Voisins trouvés")
print(f"   ✅ Similarité moyenne: {weights[:, 1:].mean():.4f}")
print(f"   ✅ Similarité min: {weights[:, 1:].min():.4f}")
print(f"   ✅ Similarité max: {weights[:, 1:].max():.4f}")


🔗 CONSTRUCTION DU GRAPHE k-NN

🔧 Normalisation des embeddings (cosine similarity)...
   ✅ Normes moyennes: 1.0000

🔄 Construction de l'index k-NN (k=10)...
   ✅ Index k-NN construit (metric=cosine)

🔍 Recherche des 10 plus proches voisins...


KeyboardInterrupt: 

In [ ]:
# Construire la liste des arêtes avec filtrage
print(f"\n🏗️  Construction des arêtes (seuil={SIMILARITY_THRESHOLD})...")

edges = []
for i in tqdm(range(indices.shape[0]), desc="Building edges"):
    for j in range(1, indices.shape[1]):  # Skip self (j=0)
        neighbor_idx = int(indices[i, j])
        similarity = float(weights[i, j])
        
        # Filtrer par seuil de similarité
        if similarity >= SIMILARITY_THRESHOLD:
            edges.append((i, neighbor_idx, similarity))

print(f"\n✅ {len(edges):,} arêtes créées")
print(f"   Arêtes par nœud (moyenne): {len(edges) / len(articles_df):.2f}")

In [ ]:
# Créer le graphe NetworkX
print("\n🔨 Création du graphe NetworkX...")

G = nx.Graph()

# Ajouter les nœuds avec attributs
print("   Ajout des nœuds...")
for i, row in articles_df.iterrows():
    G.add_node(i, 
               title=str(row.get('title', '')),
               abstract=str(row.get('abstract', ''))[:500],  # Limiter taille
               source=str(row.get('source_x', '')),
               year=int(row.get('year', 0)) if pd.notna(row.get('year')) else 0)

# Ajouter les arêtes avec poids
print("   Ajout des arêtes...")
for i, j, weight in tqdm(edges, desc="Adding edges"):
    if i != j:  # Éviter self-loops
        if G.has_edge(i, j):
            # Garder le poids maximum si arête existe déjà
            if weight > G[i][j]['weight']:
                G[i][j]['weight'] = weight
        else:
            G.add_edge(i, j, weight=weight)

print(f"\n✅ Graphe créé:")
print(f"   Nœuds: {G.number_of_nodes():,}")
print(f"   Arêtes: {G.number_of_edges():,}")
print(f"   Densité: {nx.density(G):.6f}")

## 🧩 3. Détection des Communautés

In [ ]:
print("\n" + "="*70)
print("🧩 DÉTECTION DES COMMUNAUTÉS (Louvain)")
print("="*70)

print("\n🔄 Exécution de l'algorithme de Louvain...")
partition = community_louvain.best_partition(G, weight='weight', random_state=42)

# Ajouter les communautés au dataframe
communities = pd.Series(partition).sort_index().values
articles_df['community'] = communities

# Statistiques
n_communities = int(articles_df['community'].nunique())
community_sizes = articles_df['community'].value_counts().sort_values(ascending=False)

print(f"\n✅ {n_communities} communautés détectées")
print(f"\n📊 Distribution des tailles:")
print(f"   Taille moyenne: {community_sizes.mean():.1f} articles")
print(f"   Médiane: {community_sizes.median():.0f} articles")
print(f"   Plus grande: {community_sizes.max():,} articles")
print(f"   Plus petite: {community_sizes.min()} articles")

print(f"\n🏆 Top 10 des communautés:")
for comm_id, size in community_sizes.head(10).items():
    print(f"   Community {comm_id}: {size:,} articles ({size/len(articles_df)*100:.1f}%)")

In [ ]:
# Calculer la modularité
modularity = community_louvain.modularity(partition, G, weight='weight')
print(f"\n📈 Modularité: {modularity:.4f}")
if modularity > 0.4:
    print("   ✅ Excellente structure communautaire")
elif modularity > 0.3:
    print("   ✅ Bonne structure communautaire")
else:
    print("   ⚠️  Structure communautaire faible")

## 📊 4. Analyse Topologique Avancée

In [ ]:
print("\n" + "="*70)
print("📊 ANALYSE TOPOLOGIQUE")
print("="*70)

# Métriques de base
print("\n🔢 Métriques de base:")
n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()
density = nx.density(G)

print(f"   Nœuds: {n_nodes:,}")
print(f"   Arêtes: {n_edges:,}")
print(f"   Densité: {density:.6f}")
print(f"   Type: {'Connexe' if nx.is_connected(G) else 'Déconnecté'}")

# Composantes connexes
print("\n🔗 Composantes connexes:")
components = list(nx.connected_components(G))
print(f"   Nombre: {len(components)}")
print(f"   Plus grande: {len(max(components, key=len)):,} nœuds ({len(max(components, key=len))/n_nodes*100:.1f}%)")

if len(components) > 1:
    component_sizes = sorted([len(c) for c in components], reverse=True)
    print(f"   Top 5 tailles: {component_sizes[:5]}")

In [ ]:
# Distribution des degrés
print("\n📈 Distribution des degrés:")
degrees = dict(G.degree())
degree_values = list(degrees.values())

print(f"   Degré moyen: {np.mean(degree_values):.2f}")
print(f"   Degré médian: {np.median(degree_values):.0f}")
print(f"   Degré min: {min(degree_values)}")
print(f"   Degré max: {max(degree_values)}")
print(f"   Std: {np.std(degree_values):.2f}")

# Nœuds les plus connectés
top_nodes = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:5]
print(f"\n🌟 Top 5 nœuds par degré:")
for node_id, degree in top_nodes:
    title = G.nodes[node_id].get('title', 'N/A')[:60]
    print(f"   Node {node_id}: {degree} connexions - {title}...")

In [ ]:
# Clustering coefficient
print("\n🔺 Coefficient de clustering:")
avg_clustering = nx.average_clustering(G, weight='weight')
print(f"   Moyenne: {avg_clustering:.4f}")

if avg_clustering > 0.5:
    print("   ✅ Fort clustering (structure hiérarchique)")
elif avg_clustering > 0.3:
    print("   ✅ Clustering modéré")
else:
    print("   ⚠️  Faible clustering")

In [ ]:
# Distribution des poids des arêtes
print("\n⚖️  Distribution des poids des arêtes:")
edge_weights = [data['weight'] for _, _, data in G.edges(data=True)]

print(f"   Poids moyen: {np.mean(edge_weights):.4f}")
print(f"   Poids médian: {np.median(edge_weights):.4f}")
print(f"   Poids min: {min(edge_weights):.4f}")
print(f"   Poids max: {max(edge_weights):.4f}")

## 💾 5. Sauvegarde des Résultats

In [ ]:
print("\n" + "="*70)
print("💾 SAUVEGARDE DES RÉSULTATS")
print("="*70)

# 1. Sauvegarder le graphe NetworkX
graph_path = OUTPUT_PATH / 'article_graph.gpickle'
nx.write_gpickle(G, graph_path)
print(f"\n✅ Graphe NetworkX: {graph_path}")
print(f"   Taille: {graph_path.stat().st_size / 1024**2:.2f} MB")

# 2. Sauvegarder les articles avec communautés
csv_path = OUTPUT_PATH / 'articles_with_communities.csv'
articles_df.to_csv(csv_path, index=False)
print(f"\n✅ Articles + communautés: {csv_path}")
print(f"   {len(articles_df):,} articles avec {n_communities} communautés")

# 3. Sauvegarder les métadonnées
metadata = {
    'graph_stats': {
        'n_nodes': n_nodes,
        'n_edges': n_edges,
        'density': float(density),
        'n_communities': n_communities,
        'modularity': float(modularity),
        'avg_clustering': float(avg_clustering),
        'avg_degree': float(np.mean(degree_values))
    },
    'construction_params': {
        'k_neighbors': K_NEIGHBORS,
        'similarity_threshold': SIMILARITY_THRESHOLD,
        'use_cosine': USE_COSINE,
        'embedding_dim': embeddings.shape[1]
    },
    'community_stats': {
        'n_communities': n_communities,
        'avg_size': float(community_sizes.mean()),
        'median_size': float(community_sizes.median()),
        'max_size': int(community_sizes.max()),
        'min_size': int(community_sizes.min())
    }
}

metadata_path = OUTPUT_PATH / 'graph_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"\n✅ Métadonnées: {metadata_path}")

In [ ]:
# 4. Export pour PyTorch Geometric (GNN)
print("\n🔥 Export pour PyTorch Geometric...")

# Créer edge_index (format COO)
edge_list = list(G.edges())
edge_index = np.array([[e[0], e[1]] for e in edge_list] + 
                      [[e[1], e[0]] for e in edge_list])  # Bidirectionnel
edge_index = edge_index.T  # Shape: (2, num_edges)

# Créer edge_attr (poids)
edge_weights_list = [G[e[0]][e[1]]['weight'] for e in edge_list]
edge_attr = np.array(edge_weights_list + edge_weights_list)  # Bidirectionnel

# Sauvegarder
pyg_data = {
    'edge_index': edge_index,
    'edge_attr': edge_attr,
    'x': embeddings,  # Node features
    'y': communities,  # Node labels (communautés)
    'num_nodes': n_nodes
}

pyg_path = OUTPUT_PATH / 'graph_pyg_format.pkl'
with open(pyg_path, 'wb') as f:
    pickle.dump(pyg_data, f)

print(f"✅ Format PyG: {pyg_path}")
print(f"   edge_index shape: {edge_index.shape}")
print(f"   edge_attr shape: {edge_attr.shape}")
print(f"   x shape: {embeddings.shape}")
print(f"   y shape: {communities.shape}")

## 📊 6. Visualisations Professionnelles

In [ ]:
print("\n" + "="*70)
print("📊 VISUALISATIONS")
print("="*70)

# Créer la figure
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Distribution des degrés
ax1 = fig.add_subplot(gs[0, 0])
ax1.hist(degree_values, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax1.set_xlabel('Degré', fontsize=11)
ax1.set_ylabel('Nombre de nœuds', fontsize=11)
ax1.set_title('Distribution des Degrés', fontsize=12, fontweight='bold')
ax1.axvline(np.mean(degree_values), color='red', linestyle='--', 
            label=f'Moyenne: {np.mean(degree_values):.1f}')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Distribution des poids d'arêtes
ax2 = fig.add_subplot(gs[0, 1])
ax2.hist(edge_weights, bins=50, edgecolor='black', alpha=0.7, color='coral')
ax2.set_xlabel('Poids de similarité', fontsize=11)
ax2.set_ylabel('Nombre d\'arêtes', fontsize=11)
ax2.set_title('Distribution des Poids d\'Arêtes', fontsize=12, fontweight='bold')
ax2.axvline(SIMILARITY_THRESHOLD, color='red', linestyle='--',
            label=f'Seuil: {SIMILARITY_THRESHOLD}')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Tailles des communautés
ax3 = fig.add_subplot(gs[0, 2])
top_comms = community_sizes.head(15)
ax3.barh(range(len(top_comms)), top_comms.values, color='mediumseagreen', alpha=0.8)
ax3.set_yticks(range(len(top_comms)))
ax3.set_yticklabels([f'Comm {i}' for i in top_comms.index])
ax3.set_xlabel('Nombre d\'articles', fontsize=11)
ax3.set_title('Top 15 Communautés', fontsize=12, fontweight='bold')
ax3.invert_yaxis()
ax3.grid(True, alpha=0.3, axis='x')

# 4. Distribution log des degrés
ax4 = fig.add_subplot(gs[1, 0])
ax4.hist(degree_values, bins=50, edgecolor='black', alpha=0.7, color='purple')
ax4.set_xlabel('Degré', fontsize=11)
ax4.set_ylabel('Nombre de nœuds (log)', fontsize=11)
ax4.set_yscale('log')
ax4.set_title('Distribution des Degrés (échelle log)', fontsize=12, fontweight='bold')
ax4.grid(True, alpha=0.3)

# 5. Statistiques globales
ax5 = fig.add_subplot(gs[1, 1])
ax5.axis('off')
stats_text = f"""
STATISTIQUES DU GRAPHE
{'='*40}

📊 Structure:
   • Nœuds: {n_nodes:,}
   • Arêtes: {n_edges:,}
   • Densité: {density:.6f}
   • Degré moyen: {np.mean(degree_values):.2f}

🧩 Communautés:
   • Nombre: {n_communities}
   • Taille moyenne: {community_sizes.mean():.1f}
   • Modularité: {modularity:.4f}

🔺 Clustering:
   • Coefficient: {avg_clustering:.4f}

⚖️  Poids des arêtes:
   • Moyenne: {np.mean(edge_weights):.4f}
   • Médiane: {np.median(edge_weights):.4f}
"""
ax5.text(0.1, 0.5, stats_text, fontsize=11, verticalalignment='center',
         family='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

# 6. Distribution cumulative des degrés
ax6 = fig.add_subplot(gs[1, 2])
sorted_degrees = np.sort(degree_values)
cumulative = np.arange(1, len(sorted_degrees) + 1) / len(sorted_degrees)
ax6.plot(sorted_degrees, cumulative, linewidth=2, color='darkblue')
ax6.set_xlabel('Degré', fontsize=11)
ax6.set_ylabel('Proportion cumulative', fontsize=11)
ax6.set_title('Distribution Cumulative des Degrés', fontsize=12, fontweight='bold')
ax6.grid(True, alpha=0.3)
ax6.set_xlim(left=0)

# 7. Heatmap des métriques par communauté (top 20)
ax7 = fig.add_subplot(gs[2, :])
top_20_comms = community_sizes.head(20).index
comm_metrics = []
for comm_id in top_20_comms:
    nodes_in_comm = [n for n in G.nodes() if communities[n] == comm_id]
    subgraph = G.subgraph(nodes_in_comm)
    comm_metrics.append([
        len(nodes_in_comm),
        subgraph.number_of_edges(),
        nx.density(subgraph) if len(nodes_in_comm) > 1 else 0,
        np.mean([d for _, d in subgraph.degree()]) if len(nodes_in_comm) > 0 else 0
    ])

comm_metrics_df = pd.DataFrame(comm_metrics, 
                               columns=['Taille', 'Arêtes', 'Densité', 'Degré moy'],
                               index=[f'C{i}' for i in top_20_comms])

# Normaliser pour la heatmap
comm_metrics_norm = (comm_metrics_df - comm_metrics_df.min()) / (comm_metrics_df.max() - comm_metrics_df.min())
sns.heatmap(comm_metrics_norm.T, annot=comm_metrics_df.T, fmt='.0f', 
            cmap='YlOrRd', cbar_kws={'label': 'Valeur normalisée'},
            linewidths=0.5, ax=ax7)
ax7.set_title('Métriques des Top 20 Communautés', fontsize=12, fontweight='bold')
ax7.set_xlabel('Communauté', fontsize=11)
ax7.set_ylabel('Métrique', fontsize=11)

plt.suptitle('Analyse Complète du Graphe d\'Articles Scientifiques', 
             fontsize=16, fontweight='bold', y=0.995)

# Sauvegarder
viz_path = OUTPUT_PATH / 'graph_analysis.png'
plt.savefig(viz_path, dpi=300, bbox_inches='tight')
print(f"\n✅ Visualisation sauvegardée: {viz_path}")
plt.show()

print("\n✅ Visualisations terminées!")

# BONUS: Visualisation UMAP des embeddings par communauté
try:
    from umap import UMAP
    
    print("\n🗺️  Génération de la visualisation UMAP...")
    
    # Échantillonner si trop grand
    sample_size = min(3000, len(embeddings))
    if len(embeddings) > sample_size:
        indices = np.random.RandomState(42).choice(len(embeddings), sample_size, replace=False)
        emb_sample = embeddings[indices]
        comm_sample = communities[indices]
    else:
        emb_sample = embeddings
        comm_sample = communities
    
    # UMAP
    reducer = UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)
    embedding_2d = reducer.fit_transform(emb_sample)
    
    # Visualiser
    plt.figure(figsize=(14, 10))
    scatter = plt.scatter(embedding_2d[:, 0], embedding_2d[:, 1], 
                         c=comm_sample, s=15, alpha=0.6, cmap='tab20')
    plt.colorbar(scatter, label='ID Communauté')
    plt.title(f'Projection UMAP des Embeddings (colorés par communauté)\n{sample_size} articles échantillonnés',
             fontsize=14, fontweight='bold')
    plt.xlabel('UMAP 1', fontsize=12)
    plt.ylabel('UMAP 2', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    umap_path = OUTPUT_PATH / 'embeddings_umap_communities.png'
    plt.savefig(umap_path, dpi=300, bbox_inches='tight')
    print(f"✅ UMAP sauvegardé: {umap_path}")
    plt.show()
    
except ImportError:
    print("⚠️  UMAP non installé (pip install umap-learn)")

print("\n" + "="*70)
print("✅ CONSTRUCTION DU GRAPHE TERMINÉE!")
print("="*70)

print(f"\n📦 Fichiers générés dans {OUTPUT_PATH}:")
print(f"   1. article_graph.gpickle - Graphe NetworkX complet")
print(f"   2. articles_with_communities.csv - Articles + communautés")
print(f"   3. graph_metadata.json - Métadonnées et statistiques")
print(f"   4. graph_pyg_format.pkl - Format PyTorch Geometric")
print(f"   5. graph_analysis.png - Visualisations complètes")
print(f"   6. embeddings_umap_communities.png - Projection UMAP")

print(f"\n📊 Résumé final:")
print(f"   • {n_nodes:,} articles dans le graphe")
print(f"   • {n_edges:,} connexions sémantiques")
print(f"   • {n_communities} communautés détectées")
print(f"   • Modularité: {modularity:.4f}")
print(f"   • Prêt pour GNN et analyses avancées!")


   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔥 7. Exemple d'Utilisation avec PyTorch Geometric\n",
    "\n",
    "Code pour charger et utiliser le graphe avec des GNN"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Exemple de chargement pour PyTorch Geometric\n",
    "\n",
    "print(\"\\n\" + \"=\"*70)\n",
    "print(\"🔥 EXEMPLE PYTORCH GEOMETRIC\")\n",
    "print(\"=\"*70)\n",
    "\n",
    "# Code d'exemple (nécessite: pip install torch-geometric)\n",
    "example_code = \"\"\"\n",
    "import torch\n",
    "from torch_geometric.data import Data\n",
    "import pickle\n",
    "\n",
    "# 1. Charger les données\n",
    "with open('graph_outputs/graph_pyg_format.pkl', 'rb') as f:\n",
    "    pyg_data = pickle.load(f)\n",
    "\n",
    "# 2. Créer l'objet Data PyG\n",
    "data = Data(\n",
    "    x=torch.FloatTensor(pyg_data['x']),           # Node features (embeddings)\n",
    "    edge_index=torch.LongTensor(pyg_data['edge_index']),  # Edges\n",
    "    edge_attr=torch.FloatTensor(pyg_data['edge_attr']).unsqueeze(1),  # Edge weights\n",
    "    y=torch.LongTensor(pyg_data['y'])             # Labels (communities)\n",
    ")\n",
    "\n",
    "print(f\"Graph loaded: {data}\")\n",
    "print(f\"  - Nodes: {data.num_nodes}\")\n",
    "print(f\"  - Edges: {data.num_edges}\")\n",
    "print(f\"  - Features: {data.num_node_features}\")\n",
    "print(f\"  - Classes: {data.y.unique().numel()}\")\n",
    "\n",
    "# 3. Exemple de GNN simple\n",
    "from torch_geometric.nn import GCNConv, global_mean_pool\n",
    "import torch.nn.functional as F\n",
    "\n",
    "class ArticleGNN(torch.nn.Module):\n",
    "    def __init__(self, in_channels, hidden_channels, num_classes):\n",
    "        super().__init__()\n",
    "        self.conv1 = GCNConv(in_channels, hidden_channels)\n",
    "        self.conv2 = GCNConv(hidden_channels, hidden_channels)\n",
    "        self.conv3 = GCNConv(hidden_channels, num_classes)\n",
    "    \n",
    "    def forward(self, x, edge_index, edge_weight=None):\n",
    "        x = self.conv1(x, edge_index, edge_weight)\n",
    "        x = F.relu(x)\n",
    "        x = F.dropout(x, p=0.5, training=self.training)\n",
    "        \n",
    "        x = self.conv2(x, edge_index, edge_weight)\n",
    "        x = F.relu(x)\n",
    "        x = F.dropout(x, p=0.5, training=self.training)\n",
    "        \n",
    "        x = self.conv3(x, edge_index, edge_weight)\n",
    "        return F.log_softmax(x, dim=1)\n",
    "\n",
    "# 4. Initialiser le modèle\n",
    "model = ArticleGNN(\n",
    "    in_channels=data.num_node_features,\n",
    "    hidden_channels=64,\n",
    "    num_classes=data.y.unique().numel()\n",
    ")\n",
    "\n",
    "print(f\"\\nModèle créé: {model}\")\n",
    "print(f\"Nombre de paramètres: {sum(p.numel() for p in model.parameters())}\")\n",
    "\n",
    "# 5. Training loop (exemple simplifié)\n",
    "# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)\n",
    "# criterion = torch.nn.NLLLoss()\n",
    "# ...\n",
    "\"\"\"\n",
    "\n",
    "print(\"\\n📝 Code d'exemple pour GNN:\")\n",
    "print(example_code)\n",
    "\n",
    "print(\"\\n💡 Pour utiliser ce code:\")\n",
    "print(\"   1. pip install torch torch-geometric\")\n",
    "print(\"   2. Chargez graph_pyg_format.pkl\")\n",
    "print(\"   3. Adaptez l'architecture GNN à vos besoins\")\n",
    "print(\"   4. Entraînez sur les communautés ou d'autres tâches\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}